In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from pytorch_tabnet.tab_model import TabNetClassifier

In [2]:
# import data
df = pd.read_csv("./data/CEE_DATA.csv", quotechar="'")

X = df[
    [
        "Gender",
        "Caste",
        "coaching",
        "time",
        "Class_ten_education",
        "twelve_education",
        "medium",
        "Class_X_Percentage",
        "Class_XII_Percentage",
        "Father_occupation",
        "Mother_occupation",
    ]
]
Y = df["Performance"]

In [3]:
# split data
seed = 1
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=seed
)
print("Train Size Instances: ", X_train.shape[0])
print("Test Size Instances:", X_test.shape[0])

Train Size Instances:  466
Test Size Instances: 200


In [11]:
ohe = OneHotEncoder(sparse=False)
Xcoded = pd.DataFrame(ohe.fit_transform(X),columns=ohe.get_feature_names(['Gender', 'Caste', 'coaching', 'time', 'Class_ten_education',
       'twelve_education', 'medium', 'Class_X_Percentage',
       'Class_XII_Percentage', 'Father_occupation', 'Mother_occupation']))
X_train_enc, X_test_enc, Y_train_enc, Y_test_enc = train_test_split(
    Xcoded, Y, test_size=0.2, random_state=seed
)
print("Train Size Instances: ", X_train_enc.shape[0])
print("Test Size Instances:", X_test_enc.shape[0])

Train Size Instances:  532
Test Size Instances: 134


## TabNet

In [15]:
clf = TabNetClassifier() 
clf.fit(
  X_train_enc.to_numpy(), Y_train_enc,
  eval_set=[(X_test_enc.to_numpy(), Y_test_enc)]
)
preds = clf.predict(X_test_enc.to_numpy())

Device used : cpu
epoch 0  | loss: 1.79049 | val_0_accuracy: 0.31343 |  0:00:00s
epoch 1  | loss: 1.64455 | val_0_accuracy: 0.36567 |  0:00:00s
epoch 2  | loss: 1.54914 | val_0_accuracy: 0.35075 |  0:00:00s
epoch 3  | loss: 1.46995 | val_0_accuracy: 0.30597 |  0:00:00s
epoch 4  | loss: 1.38376 | val_0_accuracy: 0.35821 |  0:00:00s
epoch 5  | loss: 1.44107 | val_0_accuracy: 0.38806 |  0:00:00s
epoch 6  | loss: 1.36266 | val_0_accuracy: 0.30597 |  0:00:00s
epoch 7  | loss: 1.3466  | val_0_accuracy: 0.20896 |  0:00:01s
epoch 8  | loss: 1.28939 | val_0_accuracy: 0.24627 |  0:00:01s
epoch 9  | loss: 1.29345 | val_0_accuracy: 0.23881 |  0:00:01s
epoch 10 | loss: 1.25497 | val_0_accuracy: 0.23881 |  0:00:01s
epoch 11 | loss: 1.29011 | val_0_accuracy: 0.24627 |  0:00:01s
epoch 12 | loss: 1.25538 | val_0_accuracy: 0.27612 |  0:00:01s
epoch 13 | loss: 1.24907 | val_0_accuracy: 0.3209  |  0:00:01s
epoch 14 | loss: 1.23141 | val_0_accuracy: 0.33582 |  0:00:02s
epoch 15 | loss: 1.24667 | val_0_accu